In [2]:
import stanza
# stanza.download('en')

nlp_stanza = stanza.Pipeline('en',processors= 'tokenize,ner')
nlp_stanza.processors['ner'].get_known_tags()

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-11 00:26:52 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-11 00:26:52 INFO: Downloaded file to C:\Users\alber\stanza_resources\resources.json
2024-06-11 00:26:52 WARNING: Language en package default expects mwt, which has been added
2024-06-11 00:26:53 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

2024-06-11 00:26:53 INFO: Using device:

['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

## Comparison

In [4]:
import numpy as np
import pandas as pd

Loading the dfs

In [5]:
df_entities_spacy = pd.read_csv(r"df_entities_spacy_processed.csv")

In [6]:
df_entities_stanza = pd.read_csv(r"df_entities_stanza_processed.csv")

In [7]:
list_people = np.unique(df_entities_spacy["subject"])

In [8]:
df_entities_spacy.columns

Index(['text', 'n_word', 'start_pos', 'end_pos', 'label', 'biography',
       'subject', 'source_index', 'category', 'range_span'],
      dtype='object')

### FX

This works, just need to complete it a lil bit

In [20]:
for person in list_people:
    
    filter_df_spacy = df_entities_spacy[df_entities_spacy["subject"] == person][["subject", "text", "label", "start_pos", "range_span"]]
    filter_df_stanza = df_entities_stanza[df_entities_stanza["subject"] == person][["subject", "text", "label", "start_pos", "range_span"]]
    i = 0
    j = 0

    partial_agreement = 0
    total_agreement = 0
    label_agreement = 0

    while i+1 < len(filter_df_spacy) and j+1 < len(filter_df_stanza): # adjusting for mismatch bt indexing and len

        # print(i, j)
        token_text_spacy = filter_df_spacy.iloc[i]["text"]
        token_text_stanza = filter_df_stanza.iloc[j]["text"]
        
        # print(token_text_spacy, token_text_stanza)
        # print(len(token_text_spacy), len(token_text_stanza))
        
        token_span_spacy = filter_df_spacy.iloc[i]["range_span"].replace("[", "").replace("]", "").replace("\n", "").replace("  ", " ").strip().split(" ")
        token_span_stanza = filter_df_stanza.iloc[j]["range_span"].replace("[", "").replace("]", "").replace("\n", "").replace("  ", " ").strip().split(" ")

        # print(token_span_spacy, token_span_stanza)
        # print(len(token_span_spacy), len(token_span_stanza))

        pos_token_spacy = filter_df_spacy.iloc[i]["label"]
        pos_token_stanza = filter_df_stanza.iloc[j]["label"]

        list_agreement_span = [index for index in token_span_spacy if index in token_span_spacy and index in token_span_stanza]
        if token_span_spacy == token_span_stanza:
            # print(token_text_spacy, token_text_stanza)
            total_agreement = total_agreement + 1
            if pos_token_spacy == pos_token_stanza:
                label_agreement = label_agreement + 1
            i = i + 1
            j = j + 1
            
            
        elif len(list_agreement_span) > 1:
            # print(token_text_spacy, token_text_stanza)
            partial_agreement = partial_agreement + 1 
            if pos_token_spacy == pos_token_stanza:
                label_agreement = label_agreement + 1
                
            i = i + 1
            j = j + 1
            
        else:
            if token_span_spacy[-1] < token_span_stanza[-1]:
                i = i + 1
                j = j
            elif token_span_spacy[-1] > token_span_stanza[-1]:
                i = i
                j = j + 1
        # print()
        # print()
    total = len(filter_df_spacy) + len(filter_df_stanza) - partial_agreement - total_agreement
    print(person)
    print(" Same annotations:", total_agreement)
    print(" Partial annotations:", partial_agreement)
    print(" Total:", total)
    print(" Perc. total or partial annotations:", round(((total_agreement+partial_agreement)/total), 2))
    print()

Aage Bohr
 Same annotations: 223
 Partial annotations: 27
 Total: 328
 Perc. total or partial annotations: 0.76

Aaron Klug
 Same annotations: 110
 Partial annotations: 19
 Total: 161
 Perc. total or partial annotations: 0.8

Adolf Friedrich Johann Butenandt
 Same annotations: 19
 Partial annotations: 2
 Total: 324
 Perc. total or partial annotations: 0.06

Adolf Otto Reinhold Windaus
 Same annotations: 26
 Partial annotations: 3
 Total: 145
 Perc. total or partial annotations: 0.2

Adolf von Baeyer
 Same annotations: 120
 Partial annotations: 22
 Total: 180
 Perc. total or partial annotations: 0.79

Albert A. Michelson
 Same annotations: 360
 Partial annotations: 66
 Total: 521
 Perc. total or partial annotations: 0.82

Albert Einstein
 Same annotations: 1151
 Partial annotations: 165
 Total: 1548
 Perc. total or partial annotations: 0.85

Alexander Prokhorov
 Same annotations: 30
 Partial annotations: 4
 Total: 430
 Perc. total or partial annotations: 0.08

Alfred Kastler
 Same annot

In [14]:
total_agreement

88

INTER ANNOTATOR AGREEMENT BETWEEN THE TWO PACKAGES